# Importing Modules and CSV files

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import password
from config import user

In [2]:
world_population_data=pd.read_csv("./Resources/World_Population.csv")
Country_population_data=pd.read_csv("./Resources/Country_Population.csv",encoding= 'unicode_escape')

In [3]:
world_population_data

,Rank,Country,Region,Population,Percentage,Date
0,1,China,Asia,1411778724,17.80%,20-Nov
1,2,India,Asia,1386141732,17.50%,21-Dec
2,3,United States,Americas,332960297,4.20%,21-Dec
3,4,Indonesia,Asia,271350000,3.43%,20-Dec
4,5,Pakistan,Asia,225200000,2.84%,21-Jul
...,...,...,...,...,...,...
236,237,Niue(NewZealand),Oceania,1549,0%,21-Jul
237,238,Tokelau(NewZealand),Oceania,1501,0%,21-Jul
238,239,VaticanCity,Europe,825,0%,19-Feb
239,240,Cocos(Keeling)Islands(Australia),Oceania,573,0%,20-Jun


In [4]:
Country_population_data

,Country,Population (2020),Land Area (Km²),Density (P/Km²)
0,Afghanistan,"38,041,750","652,230",56.94
1,Albania,"2,854,190","28,748",104.61
2,Algeria,"43,053,050","2,381,740",17.73
3,Andorra,"77,140",468,163.84
4,Angola,"31,825,290","1,246,700",24.71
...,...,...,...,...
189,Venezuela,"28,515,830","912,050",32.73
190,Vietnam,"96,462,110","331,210",308.13
191,Yemen,"29,161,920","527,968",53.98
192,Zambia,"17,861,030","752,618",23.34


# Data Cleaning

In [5]:
Country_population_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            194 non-null    object 
 1   Population (2020)  194 non-null    object 
 2   Land Area (Km²)    194 non-null    object 
 3   Density (P/Km²)    194 non-null    float64
dtypes: float64(1), object(3)
memory usage: 6.2+ KB


In [6]:
world_population_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rank        241 non-null    int64 
 1   Country     241 non-null    object
 2   Region      241 non-null    object
 3   Population  241 non-null    int64 
 4   Percentage  241 non-null    object
 5   Date        241 non-null    object
dtypes: int64(2), object(4)
memory usage: 11.4+ KB


In [7]:
world_population_data.drop("Rank",inplace=True,axis=1)

In [8]:
Country_population_data.drop("Population (2020)",inplace=True,axis=1)

In [9]:
world_population_data = world_population_data.rename(columns={'Country': 'country', 'Region': 'region','Population': 'population','Percentage':'percentage','Date': 'date'})

In [10]:
Country_population_data = Country_population_data.rename(columns={'Country': 'country', 'Land Area (Km²)': 'land_area','Density (P/Km²)':'density'})

In [11]:
world_population_data

,country,region,population,percentage,date
0,China,Asia,1411778724,17.80%,20-Nov
1,India,Asia,1386141732,17.50%,21-Dec
2,United States,Americas,332960297,4.20%,21-Dec
3,Indonesia,Asia,271350000,3.43%,20-Dec
4,Pakistan,Asia,225200000,2.84%,21-Jul
...,...,...,...,...,...
236,Niue(NewZealand),Oceania,1549,0%,21-Jul
237,Tokelau(NewZealand),Oceania,1501,0%,21-Jul
238,VaticanCity,Europe,825,0%,19-Feb
239,Cocos(Keeling)Islands(Australia),Oceania,573,0%,20-Jun


In [12]:
population_count_before=world_population_data["population"].sum()

In [13]:
Country_population_data["land_area"] = Country_population_data["land_area"].str.replace(",", "")

In [14]:
Country_population_data

,country,land_area,density
0,Afghanistan,652230,56.94
1,Albania,28748,104.61
2,Algeria,2381740,17.73
3,Andorra,468,163.84
4,Angola,1246700,24.71
...,...,...,...
189,Venezuela,912050,32.73
190,Vietnam,331210,308.13
191,Yemen,527968,53.98
192,Zambia,752618,23.34


# Create Database Connection

In [15]:
protocol="postgresql"
username=user
password=password
host="localhost"
port=5432
database_name="World_Pop"
connection_string=f"{protocol}://{username}:{password}@{host}:{port}/{database_name}"

In [16]:
# Confirm tables
engine=create_engine(connection_string)
engine.table_names()

C:\Users\rahmi\AppData\Local\Temp\ipykernel_6328\845267981.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['worldpopulations', 'countrypopulations']

In [17]:
world_population_data.to_sql(name="worldpopulations",con=engine,if_exists="append", index=False)

241

In [18]:
Country_population_data.to_sql(name="countrypopulations",con=engine,if_exists="append", index=False)

194

In [26]:
merged=pd.read_sql("select * from merged", con=engine)
merged

,country,region,population,percentage,date,land_area,density
0,China,Asia,1411778724,17.80%,20-Nov,9596960.0,147.77
1,India,Asia,1386141732,17.50%,21-Dec,3287263.0,454.94
2,Indonesia,Asia,271350000,3.43%,20-Dec,1904375.0,142.56
3,Pakistan,Asia,225200000,2.84%,21-Jul,796095.0,275.29
4,Brazil,Americas,214143381,2.70%,21-Dec,8515770.0,25.06
...,...,...,...,...,...,...,...
158,Liechtenstein,Europe,39151,0.00%,21-Jun,160.0,236.94
159,Monaco,Europe,38350,0.00%,20-Dec,2.0,19083.37
160,Palau,Oceania,17957,0.00%,21-Jul,459.0,38.93
161,Nauru,Oceania,11832,0.00%,21-Jul,21.0,635.20


In [27]:
merged={"Country":merged['country'],
                         "Region":merged['region'],
        "Population":merged['population'],
        "Percentage":merged['percentage'],
        "Date":merged['date'],
        "Land Area":merged['land_area'],
        "Density":merged['density']
}
merged=pd.DataFrame(merged)
merged

,Country,Region,Population,Percentage,Date,Land Area,Density
0,China,Asia,1411778724,17.80%,20-Nov,9596960.0,147.77
1,India,Asia,1386141732,17.50%,21-Dec,3287263.0,454.94
2,Indonesia,Asia,271350000,3.43%,20-Dec,1904375.0,142.56
3,Pakistan,Asia,225200000,2.84%,21-Jul,796095.0,275.29
4,Brazil,Americas,214143381,2.70%,21-Dec,8515770.0,25.06
...,...,...,...,...,...,...,...
158,Liechtenstein,Europe,39151,0.00%,21-Jun,160.0,236.94
159,Monaco,Europe,38350,0.00%,20-Dec,2.0,19083.37
160,Palau,Oceania,17957,0.00%,21-Jul,459.0,38.93
161,Nauru,Oceania,11832,0.00%,21-Jul,21.0,635.20


## Finding the most densely populated regions in the world

In [23]:
merged_group = merged.groupby(["Region"]).sum()
merged_group.sort_values(by="Density",ascending=False)
merged_group.drop(columns="Land Area")

,Population,Density
Region,,
Africa,1108765102,4738.68
Americas,675188022,3130.42
Asia,4351527869,18267.58
"Asia,Europe",83614362,106.96
Europe,582141838,25216.29
"Europe,Asia",146171015,8.82
Oceania,28416934,1673.23


## Finding most populated regions in the world

In [25]:
merged_group.sort_values(by="Population",ascending=False)

,Population,Land Area,Density
Region,,,
Asia,4351527869,30098827.0,18267.58
Africa,1108765102,24452242.0,4738.68
Americas,675188022,30349930.0,3130.42
Europe,582141838,5834838.0,25216.29
"Europe,Asia",146171015,17098242.0,8.82
"Asia,Europe",83614362,783562.0,106.96
Oceania,28416934,7806176.0,1673.23
